In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
train_df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
train_df.head()


In [ ]:
# Checking for missing values
train_df.isnull().sum()


In [ ]:
# Dropping rows with missing keywords or locations 
train_df = train_df.dropna(subset=['keyword', 'location'])

# Reseting index after dropping
train_df.reset_index(drop=True, inplace=True)

# Checking new shape of the data
train_df.shape


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting the target distribution
sns.countplot(x='target', data=train_df)
plt.title('Distribution of Disaster vs Non-Disaster Tweets')
plt.xlabel('Target (0 = Non-disaster, 1 = Disaster)')
plt.ylabel('Count')
plt.show()

# Displaying counts as text
print(train_df['target'].value_counts())


In [ ]:
# How many unique keywords?
print("Number of unique keywords:", train_df['keyword'].nunique())

# Most frequent keywords
top_keywords = train_df['keyword'].value_counts().head(15)

# Plotting top keywords
plt.figure(figsize=(10,6))
sns.barplot(y=top_keywords.index, x=top_keywords.values)
plt.title('Top 15 Most Frequent Keywords')
plt.xlabel('Frequency')
plt.ylabel('Keyword')
plt.show()


In [ ]:
# Top 15 locations
top_locations = train_df['location'].value_counts().head(15)

plt.figure(figsize=(10,6))
sns.barplot(y=top_locations.index, x=top_locations.values)
plt.title('Top 15 Tweet Locations')
plt.xlabel('Frequency')
plt.ylabel('Location')
plt.show()


In [ ]:
# Creating new features
train_df['text_length'] = train_df['text'].apply(len)
train_df['word_count'] = train_df['text'].apply(lambda x: len(x.split()))

# Plotting tweet length distribution by class
plt.figure(figsize=(10,6))
sns.histplot(data=train_df, x='text_length', hue='target', bins=40, kde=True)
plt.title('Tweet Length Distribution by Target')
plt.xlabel('Text Length (characters)')
plt.ylabel('Count')
plt.show()


In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Downloading stopwords if not already done
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean_text(text):
    text = text.lower()                                # lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # removing URLs
    text = re.sub(r'\@w+|\#','', text)                 # removing mentions and hashtags
    text = re.sub(r'\d+', '', text)                    # removing numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # removing punctuation
    words = text.split()
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    return " ".join(words)

# Applying cleaning
train_df['clean_text'] = train_df['text'].apply(clean_text)


In [ ]:
train_df[['text', 'clean_text']].head(5)


In [ ]:
from wordcloud import WordCloud

# Separating clean texts by class
disaster_tweets = train_df[train_df['target'] == 1]['clean_text']
non_disaster_tweets = train_df[train_df['target'] == 0]['clean_text']

# Generating WordClouds
plt.figure(figsize=(12,6))
wordcloud1 = WordCloud(width=600, height=400, background_color='white').generate(" ".join(disaster_tweets))
plt.subplot(1, 2, 1)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis('off')
plt.title('Disaster Tweets WordCloud')

wordcloud0 = WordCloud(width=600, height=400, background_color='white').generate(" ".join(non_disaster_tweets))
plt.subplot(1, 2, 2)
plt.imshow(wordcloud0, interpolation='bilinear')
plt.axis('off')
plt.title('Non-Disaster Tweets WordCloud')
plt.show()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initializing TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=5000)  # You can try 10,000 later

# Fit on training clean text and transform
X = tfidf.fit_transform(train_df['clean_text'])

# Targeting labels
y = train_df['target']


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Splitting into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Training logistic regression
model = LogisticRegression()
model.fit(X_train, y_train)

# Predicting on validation set
y_pred = model.predict(X_val)

# Evaluating
print("Validation Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred))


In [ ]:
# Cleaning test data using the same function
test_df['clean_text'] = test_df['text'].apply(clean_text)

# Transforming test data using the same TF-IDF (never fit again!)
X_test = tfidf.transform(test_df['clean_text'])

# Making predictions
test_preds = model.predict(X_test)

# Preparing submission DataFrame
submission = pd.DataFrame({
    'id': test_df['id'],
    'target': test_preds
})

# Saving to CSV
submission.to_csv('submission.csv', index=False)
